In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import umap

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib.image import imread

from PIL import Image,ImageOps

from sklearn.decomposition import PCA

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

import torch

In [ ]:
import matplotlib.pyplot as plt 
plt.rcParams.update({
    "font.family" : 'Arial',   # specify font family here
    "font.size" : 12,
    "font.weight" : "normal"})          # specify font size here

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
base_path = '/home/jinny/projects/Art-history/Art-history/datas/'
file_info = pd.read_csv(base_path+'file_info.csv')

# file_info_latent
df = pd.DataFrame(( np.load(base_path+'vectors/avec_latents.npy', allow_pickle=True)),columns=['avec','Path'])
file_info_latents = pd.merge(file_info, df, how = 'left', on = 'Path')
file_info_latents = file_info_latents[~file_info_latents.avec.isnull()]

avec = np.array([i.reshape(-1) for i in file_info_latents['avec']])

In [ ]:
df = pd.DataFrame(np.load(base_path+'vectors/cvec_latents.npy', allow_pickle=True),columns=['cvec','Path'])
file_info_latents = pd.merge(file_info_latents, df, how = 'left', on = 'Path')

cvec = np.array([i.reshape(-1) for i in file_info_latents['cvec']])

In [ ]:
pca_avec = PCA(n_components=5,random_state=42)
pca_fit = pca_avec.fit_transform(avec)
pca_avec_df = pd.DataFrame(data=pca_fit)

pca_avec_df.columns = ['avec_pc'+str(i+1) for i in range(5)]

In [ ]:
pca_cvec = PCA(n_components=5,random_state=42)
pca_fit = pca_cvec.fit_transform(cvec)
pca_cvec_df = pd.DataFrame(data=pca_fit)

pca_cvec_df.columns = ['cvec_pc'+str(i+1) for i in range(5)]

In [ ]:
file_info_latents = pd.concat([file_info_latents,pca_avec_df.iloc[:,:5]],axis=1)
file_info_latents = pd.concat([file_info_latents,pca_cvec_df.iloc[:,:5]],axis=1)

In [ ]:
temp_file_info_latents = file_info_latents[file_info_latents['new_date']>=1500].reset_index(drop=True)

### graph-a

In [ ]:
def make_pca_ridge(pc, vec, ticks):
    sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

    # new_date 값을 내림차순으로 정렬
    temp_file_info_latents_sorted = temp_file_info_latents.sort_values(by="new_date", ascending=False)
    unique_dates = sorted(temp_file_info_latents["new_date"].unique())

    # FacetGrid를 사용하여 데이터 분리 (내림차순 정렬된 데이터 사용)
    g = sns.FacetGrid(
        temp_file_info_latents,
        row="new_date",
        hue="new_date",
        aspect=30,
        height=0.2,
        palette="Spectral_r",
        row_order=unique_dates,
    )
    years = list(temp_file_info_latents["new_date"].unique())

    # 각 능선에 KDE 플롯 추가
    g.map(sns.kdeplot, f"{vec}vec_pc{pc}", clip_on=False, shade=True, alpha=1, lw=1, bw_adjust=0.5)
    g.map(sns.kdeplot, f"{vec}vec_pc{pc}", clip_on=False, color="w", lw=1.5, bw_adjust=0.5)

    # 각 능선의 오버랩을 조정
    g.map(plt.axhline, y=0, lw=2, clip_on=False)

    # 그래프의 각 요소를 디자인
    g.fig.subplots_adjust(hspace=-0.6)

    # 범례 제거 및 기본 설정
    g.set_titles("")
    g.set(yticks=[], ylabel=None, xlabel=None)
    g.despine(bottom=True, left=True)

    # # x축 수치 표시 - 마지막 subplot에만 적용
    # for ax in g.axes[:-1].flat:
    #     ax.set_xticks([])
    
    bottom_ax = g.axes[-1, 0]
    bottom_ax.set_xticks(ticks,fontsize=25)  # x축 눈금을 설정
    bottom_ax.set_xticklabels(ticks, fontsize=25)


    # Colorbar 추가
    norm = Normalize(
        vmin=min([years.index(x) for x in temp_file_info_latents["new_date"]]),
        vmax=max([years.index(x) for x in temp_file_info_latents["new_date"]]),
    )
    cbar_ax = g.fig.add_axes([0.95, 0.1, 0.03, 0.75])
    sm = ScalarMappable(cmap="Spectral_r", norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(
        sm, ax=g.axes.ravel().tolist(), cax=cbar_ax, orientation="vertical", aspect=30, fraction=0.05
    )
    cbar.set_ticks([0, 25, 49])
    cbar.set_ticklabels([1500, 1750, 1990], fontsize=36)
    cbar.ax.invert_yaxis()

    # 제목 추가
    g.fig.suptitle(f"PC{pc}", fontsize=46, y=0.95, x=0.6)  # y로 타이틀 위치 조정
    plt.savefig(
        f"/home/jinny/projects/Art-history/Art-history/graph/figure02_AB_{vec}vec_pc{pc}.svg",
        bbox_inches="tight",
        transparent=True,
        dpi=600,
    )
    plt.show()


In [ ]:
make_pca_ridge(1,'a',range(-160,161,80))
make_pca_ridge(2,'a',range(-110,111,55))
make_pca_ridge(1,'c',np.arange(-0.5,0.51,0.25))
make_pca_ridge(2,'c',np.arange(-0.5,0.51,0.25))

### graph-b

In [ ]:
import os
os.chdir("./scripts")

In [ ]:
import argparse, os
import PIL
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange, repeat
from torchvision.utils import make_grid
from torch import autocast
from contextlib import nullcontext
from pytorch_lightning import seed_everything
from imwatermark import WatermarkEncoder
import pandas as pd
from scripts.txt2img import put_watermark
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib.image import imread

In [ ]:
base_path = '/home/jinny/projects/Art-history/Art-history/'

In [ ]:
munch = file_info_latents[file_info_latents['author_name']=='edvard munch']
munch = munch[munch['painting_name']=='The Scream 1895']

m_path = munch['Path'].values[0]
m_latent = munch['avec'].values[0]

In [ ]:
pearl = file_info_latents[file_info_latents['author_name'].str.contains('vermeer')]
pearl = pearl[pearl['painting_name']=='The Girl With A Pearl Earring']

p_path = pearl['Path'].values[0]
p_latent = pearl['avec'].values[0]

In [ ]:
avec_eigen_variance = pca_avec.explained_variance_ratio_
avec_eigen_vector = pca_avec.components_
avec_eigen_value  = pca_avec.singular_values_ 

In [ ]:
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model

def load_model(cudanum) :

    config = OmegaConf.load(base_path+f"scripts/configs/stable-diffusion/v2-inference.yaml")
    model = load_model_from_config(config, base_path+f"scripts/configs/checkpoint/512-base-ema.ckpt") 

    GPU_NUM=cudanum
    
    # GPU 할당 변경하기
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device) # change allocation of current GPU
    print ('Current cuda device ', torch.cuda.current_device()) # check

    model = model.to(device)
    return model,device

def load_img(path):
    image = Image.open(path).convert("RGB")
    w, h = image.size
    print(f"loaded input image of size ({w}, {h}) from {path}")
    w, h = map(lambda x: x - x % 64, (w, h))  # resize to integer multiple of 64
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2. * image - 1.

In [ ]:
def run_diffusion_model_changing_denoising_steps(init_latent,prompt,n,num,batch,model,device):  
    
    ddim_steps = 50
    ddim_eta = 0.0
    n_iter = 1
    batch_size = batch
    scale = 9.0
    
    sampler = DDIMSampler(model)
    sampler.make_schedule(ddim_num_steps=ddim_steps, ddim_eta=ddim_eta, verbose=False)
    precision_scope = autocast
    
    all_samples = []
    all_samples_enc = []
    for i in range(n):
        #strength = 0.8
        strength = 1/ddim_steps * (i+num) # change strength to control denoising steps
        data = [batch_size * [prompt]]
        t_enc = int(strength * ddim_steps)
        sampler.make_schedule(ddim_num_steps=ddim_steps, ddim_eta=ddim_eta, verbose=False)
        with torch.no_grad():
            with precision_scope("cuda"):
                with model.ema_scope():
                    for n in trange(n_iter, desc="Sampling"):
                        for prompts in tqdm(data, desc="data"):
                            uc = None
                            if scale != 1.0:
                                uc = model.get_learned_conditioning(batch_size * [""])
                            if isinstance(prompts, tuple):
                                prompts = list(prompts)
                            c = model.get_learned_conditioning(prompts)
                            if prompt == "no prompt" :
                                print("change prompt vector to zero")
                                c = torch.zeros(c.shape[0], c.shape[1], c.shape[2]).to('cuda:0') # change prompt vector to zero (i.e., no direction)

                            # encode (scaled latent)
                            z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc] * batch_size).to(device))
                            # decode it
                            samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=scale,
                                                     unconditional_conditioning=uc, )

                            x_samples = model.decode_first_stage(samples)
                            x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
                            
        #draw_image(x_samples[0])
        all_samples.append(x_samples)
        all_samples_enc.append(samples)
               
    return all_samples_enc

In [ ]:
def model_2latent(path,model) :
    file_info = pd.read_csv(base_path+'datas/file_info.csv')

    image_latent = []
    for i in path :
        init_image = load_img(base_path+'datas/resized_image/'+i).to(device)
        init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))
        init_latent = init_latent.view([-1])
        image_latent.append(init_latent.cpu().numpy())

    #latent_df = pd.DataFrame(image_latent)
    return image_latent

def model_2img(latent_samples, model) :
    precision_scope = autocast 
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                
                x_samples = model.decode_first_stage(latent_samples)
                x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                for x_sample in x_samples:
                    x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                    img = Image.fromarray(x_sample.astype(np.uint8))
                    #img = put_watermark(img, wm_encoder)
                    return img
                    # plt.imshow(img)
                    # plt.show()

In [ ]:
# model load
model,device = load_model(5)

In [ ]:
def move_vector_along_pca_axis(vector, pca_axis, distance):
    # 벡터를 복사하여 원본 벡터를 변경하지 않도록 함
    moved_vector = np.copy(vector)
    normalized_pca_axis = np.copy(pca_axis)
    

    normalized_pca_axis = normalized_pca_axis / np.linalg.norm(normalized_pca_axis)
    # PCA 축 벡터에 특정 거리만큼 스케일링하여 이동
    moved_vector += distance * normalized_pca_axis

    return moved_vector

In [ ]:
def make_img(pc_num) :
    change_latnets1 = list()
    # original
    change_latnet = move_vector_along_pca_axis(m_latent,avec_eigen_vector[pc_num],0)
    change_latnet = torch.Tensor(change_latnet.reshape(1,4,64,64)).cuda()
    change_latnets1.append(change_latnet)
    # min
    change_latnet = move_vector_along_pca_axis(m_latent,avec_eigen_vector[pc_num],-pca_avec_df[f'avec_pc{pc_num+1}'][munch.index.values[0]]+min(pca_avec_df[f'avec_pc{pc_num+1}']))
    change_latnet = torch.Tensor(change_latnet.reshape(1,4,64,64)).cuda()
    change_latnets1.append(change_latnet)
    # max
    change_latnet = move_vector_along_pca_axis(m_latent,avec_eigen_vector[pc_num],-pca_avec_df[f'avec_pc{pc_num+1}'][munch.index.values[0]]+max(pca_avec_df[f'avec_pc{pc_num+1}']))
    change_latnet = torch.Tensor(change_latnet.reshape(1,4,64,64)).cuda()
    change_latnets1.append(change_latnet)

    # move 
    change_latnets2 = list()
    for i in range(13) :
        change_latnet = move_vector_along_pca_axis(m_latent,avec_eigen_vector[pc_num],-pca_avec_df[f'avec_pc{pc_num+1}'][munch.index.values[0]]-300+i*50)
        change_latnet = torch.Tensor(change_latnet.reshape(1,4,64,64)).cuda()
        change_latnets2.append(change_latnet)
    return change_latnets1,change_latnets2

In [ ]:
def plot_image_on_graph(x_coord,y_coord, change_latnet, ax,color,border):
   # 이미지 불러오기
    img = model_2img(change_latnet, model)

    # 이미지 크기 조정
    img = img.resize((80, 80))
    img = ImageOps.expand(img, border=border, fill=color)

    # 그래프 좌표계에 맞춰 이미지를 배치 (imshow 사용)
    imagebox = OffsetImage(img, zoom=1)
    ab = AnnotationBbox(imagebox, (x_coord, y_coord), frameon=False, zorder=10)
    
    # 이미지 배치
    ax.add_artist(ab)

In [ ]:
pc_img_info = dict()

In [ ]:
for pc_num in range(4) :
    # 그래프 그리기
    ### 라인 그리기
    y_point = 0.76
    fig, ax = plt.subplots(figsize=(14,5))
    x_values = [-230,230]
    y_values = [y_point,y_point]
    ax.plot(x_values, y_values,color='black',zorder=1,linewidth=1)
    ### 점 그리기
    x_values = [-200+50*i for i in range(9)]
    y_values = [y_point for i in range(len(x_values))]
    ax.scatter(x_values, y_values,c='black',s=35,zorder=3)
    ax.plot([-230, -230], [y_point-0.05, y_point+0.05], color='black', linewidth=1)  # 왼쪽 막대기
    ax.plot([230, 230], [y_point-0.05, y_point+0.05], color='black', linewidth=1)    # 오른쪽 막대기
    ### min,max,original 점 그리기
    x_values = [min(pca_avec_df[f'avec_pc{pc_num+1}']), pca_avec_df[f'avec_pc{pc_num+1}'][munch.index.values[0]], pca_avec_df[f'avec_pc{pc_num+1}'][pearl.index.values[0]], max(pca_avec_df[f'avec_pc{pc_num+1}'])]
    y_values = [y_point for i in range(len(x_values))]
    ax.scatter(x_values, y_values, c='red',s=100,zorder=5)
    # munch
    ax.scatter(x_values[1], y_values[1], c='green',s=100,zorder=5)
    # pearl
    ax.scatter(x_values[2], y_values[2], c='blue',s=100,zorder=5)

    ### img path
    temp_df = pca_avec_df.copy()
    temp_df['pc-min'] = temp_df[f'avec_pc{pc_num+1}'] - min(pca_avec_df[f'avec_pc{pc_num+1}'])
    temp_df['pc-max'] = temp_df[f'avec_pc{pc_num+1}'] - max(pca_avec_df[f'avec_pc{pc_num+1}'])

    temp_df = temp_df.reset_index()

    temp_df_min = temp_df.sort_values('pc-min')
    min_path = file_info_latents['avec'][file_info_latents.index == temp_df_min['index'].iloc[0]].values[0]

    temp_df_max = temp_df.sort_values('pc-max')
    max_path = file_info_latents['avec'][file_info_latents.index == temp_df_max['index'].iloc[-1]].values[0]

    plt.xlim([-235, 235])      
    plt.ylim([-0.2, 2])
    plt.gca().axes.yaxis.set_visible(False) 
    plt.xticks(fontsize=30)

    ## sample 좌표
    sam = (pca_avec_df[f'avec_pc{pc_num+1}'][munch.index.values[0]] + pca_avec_df[f'avec_pc{pc_num+1}'][pearl.index.values[0]])/2
    text_positions = [sam-27,sam+27,min(pca_avec_df[f'avec_pc{pc_num+1}']),max(pca_avec_df[f'avec_pc{pc_num+1}'])] # 텍스트를 추가할 위치들의 x, y 좌표 리스트

    # img
    if pca_avec_df[f'avec_pc{pc_num+1}'][pearl.index.values[0]] - pca_avec_df[f'avec_pc{pc_num+1}'][munch.index.values[0]] < 60 :
        xm_coords_image = sam-27
        xp_coords_image = sam+27
    else :
        xm_coords_image = pca_avec_df[f'avec_pc{pc_num+1}'][munch.index.values[0]]
        xp_coords_image = pca_avec_df[f'avec_pc{pc_num+1}'][pearl.index.values[0]]
        text_positions[0] = xm_coords_image
        text_positions[1] = xp_coords_image    
        
    y_coords_image = 1.5

    x1_coords_images = [i for i in [min(pca_avec_df[f'avec_pc{pc_num+1}']),max(pca_avec_df[f'avec_pc{pc_num+1}'])] ]
    y1_coords_images = [y_coords_image for i in range(len(x1_coords_images))]

    x2_coords_images = [(i*50)-200 for i in range(9)]
    y2_coords_images = [0.3 for i in range(len(x2_coords_images))]

    # 텍스트 쓰기
    text_y = 0.98
    texts = ['(a)','(b)', 'min','max']  # 추가할 텍스트 내용 리스트
    ax.text(text_positions[0], text_y, texts[0], fontsize=38, ha='center', va='center', color='green')
    ax.text(text_positions[1], text_y, texts[1], fontsize=38, ha='center', va='center', color='blue')
    ax.text(text_positions[2], text_y, texts[2], fontsize=38, ha='center', va='center', color='red')
    ax.text(text_positions[3], text_y, texts[3], fontsize=38, ha='center', va='center', color='red')

    result_minmax = [torch.tensor(min_path.reshape(1,4,64,64)).cuda(),torch.tensor(max_path.reshape(1,4,64,64)).cuda()]
    result_munch = torch.tensor(m_latent.reshape(1,4,64,64)).cuda()
    result_pearl = torch.tensor(p_latent.reshape(1,4,64,64)).cuda()

    # 이미지 표시 함수 호출
    change_latnets1,change_latnets2 = make_img(pc_num)
    plot_image_on_graph(xm_coords_image, y_coords_image, result_munch, ax,'green',3)
    plot_image_on_graph(xp_coords_image, y_coords_image, result_pearl, ax,'blue',3)
    for x, y, path in zip(x1_coords_images, y1_coords_images, result_minmax):
        plot_image_on_graph(x, y, path, ax,'red',3)
    for x, y, path in zip(x2_coords_images, y2_coords_images, change_latnets2):
        plt.subplots_adjust(top=0.9)
        plot_image_on_graph(x, y, path, ax,'black',1)

    # 그림 정보 저장
    # min, munch, pearl, max
    # (min) point, title, author, year, (max) point, title, author, year , munch, pearl
    temp_max = file_info_latents[file_info_latents.index == temp_df_max['index'].iloc[-1]]
    temp_min = file_info_latents[file_info_latents.index == temp_df_min['index'].iloc[0]]
    pc_img_info[f'A_PC{pc_num+1}'] = [min(pca_avec_df[f'avec_pc{pc_num+1}']), temp_min['painting_name'].values[0], temp_min['author_name'].values[0], temp_min['new_date'].values[0], max(pca_avec_df[f'avec_pc{pc_num+1}']), temp_max['painting_name'].values[0], temp_max['author_name'].values[0], temp_max['new_date'].values[0],
                                    pca_avec_df[f'avec_pc{pc_num+1}'][munch.index.values[0]], pca_avec_df[f'avec_pc{pc_num+1}'][pearl.index.values[0]]]

    # 그래프 보여주기
    plt.title(f'PC{pc_num+1}',fontsize=45, pad=10)
    plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/figure02_C_pc{pc_num+1}.svg',bbox_inches='tight',transparent = True,dpi=600)

    plt.show()

In [ ]:
A_vector_PC_image_info = pd.DataFrame(pc_img_info).T
A_vector_PC_image_info.columns = ['min_point', 'min_title', 'min_author', 'min_date', 'max_point', 'max_title', 'max_author', 'max_date', 'munch', 'pearl']

In [ ]:
A_vector_PC_image_info

### graph-C

In [ ]:
cvec_eigen_variance = pca_cvec.explained_variance_ratio_
cvec_eigen_vector = pca_cvec.components_
cvec_eigen_value  = pca_cvec.singular_values_ 

In [ ]:
def cos(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

def sin(a,b):
    return np.sqrt(1-cos(a,b)**2)

for i in range(5) :
    cvec_pc_dist = list()
    for latent in cvec :
        cvec_pc_dist.append(np.linalg.norm(latent) * sin(latent,cvec_eigen_vector[i]))
    file_info_latents[f'cvec_pc{i+1}_dist'] = cvec_pc_dist

In [ ]:
def c_plot_image_on_graph(x_coord,y_coord, change_latnet, ax,color,border):
   # 이미지 불러오기
    img = model_2img(change_latnet, model)

    # 이미지 크기 조정
    img = img.resize((80, 80))
    img = ImageOps.expand(img, border=border, fill=color)

    # 그래프 좌표계에 맞춰 이미지를 배치 (imshow 사용)
    imagebox = OffsetImage(img, zoom=1)
    ab = AnnotationBbox(imagebox, (x_coord, y_coord), frameon=False, zorder=10)
    
    # 이미지 배치
    ax.add_artist(ab)

In [ ]:
pc_img_info = dict()

In [ ]:
for pc_num in range(4) :
    
    fig, ax = plt.subplots(figsize=(14, 5))
    
    special_x = [file_info_latents[f'cvec_pc{pc_num+1}'].min(),file_info_latents[f'cvec_pc{pc_num+1}'].max()] 
    x = np.linspace(special_x[0]+0.015, special_x[1]-0.015, 9)  # 중간 9개의 점
    y_point = 0.75
    

    # 선
    ax.plot([special_x[0]-0.06, special_x[1]+0.06], [y_point,y_point], color='black', linewidth=1)
    # 양 끝에 세로 막대기 추가
    ax.plot([special_x[0]-0.06, special_x[0]-0.06], [y_point-0.05,y_point+0.05], color='black', linewidth=1)  # 왼쪽 막대기
    ax.plot([special_x[1]+0.06, special_x[1]+0.06], [y_point-0.05,y_point+0.05], color='black', linewidth=1)    # 오른쪽 막대기
    
    # 점
    ax.scatter(x, np.zeros_like(x)+y_point, color='black', zorder=5)
    # special 점
    ax.scatter(special_x, np.zeros_like(special_x)+y_point, color='red', zorder=5,s=100)
    # m&p
    ax.scatter(pca_cvec_df[f'cvec_pc{pc_num+1}'][munch.index.values[0]], y_point, color='green', zorder=5,s = 100)
    ax.scatter(pca_cvec_df[f'cvec_pc{pc_num+1}'][pearl.index.values[0]], y_point, color='blue', zorder=5,s = 100)
    
    
    # 범위와 눈금 설정
    plt.xlim(special_x[0]-0.07, special_x[1]+0.07)
    plt.ylim(-0.2, 2)  # y축 범위 설정 (이미지를 넣을 충분한 공간 확보)
    plt.gca().axes.yaxis.set_visible(False) 
    plt.xticks([ round(i,2) for i in np.linspace(special_x[0]+0.015, special_x[1]-0.015, 5)],fontsize=30)

    # img path
    temp_df = pca_cvec_df.copy()
    temp_df['pc-min'] = temp_df[f'cvec_pc{pc_num+1}'] - min(pca_cvec_df[f'cvec_pc{pc_num+1}'])
    temp_df['pc-max'] = temp_df[f'cvec_pc{pc_num+1}'] - max(pca_cvec_df[f'cvec_pc{pc_num+1}'])

    temp_df = temp_df.reset_index()

    temp_df_min = temp_df.sort_values('pc-min')
    min_latent = file_info_latents['avec'][file_info_latents.index == temp_df_min['index'].iloc[0]].values[0]

    temp_df_max = temp_df.sort_values('pc-max')
    max_latent = file_info_latents['avec'][file_info_latents.index == temp_df_max['index'].iloc[-1]].values[0]


    #### img
    # min
    ax.text(special_x[0], y_point+0.1, 'min', color='red', fontsize=38, ha='center')
    c_plot_image_on_graph(special_x[0],y_point+0.7, torch.tensor(min_latent.reshape(1,4,64,64)).cuda(), ax,'red',3)
    
    # a - munch
    ax.text(pca_cvec_df[f'cvec_pc{pc_num+1}'][munch.index.values[0]], y_point+0.1, '(a)', color='green', fontsize=38, ha='center')
    c_plot_image_on_graph(pca_cvec_df[f'cvec_pc{pc_num+1}'][munch.index.values[0]],y_point+0.7, torch.tensor(m_latent.reshape(1,4,64,64)).cuda(), ax,'green',3)
 
    # b - pearl
    ax.text(pca_cvec_df[f'cvec_pc{pc_num+1}'][pearl.index.values[0]], y_point+0.1, '(b)', color='blue', fontsize=38, ha='center')
    c_plot_image_on_graph(pca_cvec_df[f'cvec_pc{pc_num+1}'][pearl.index.values[0]],y_point+0.7, torch.tensor(p_latent.reshape(1,4,64,64)).cuda(), ax,'blue',3)
 
    # max
    ax.text(special_x[1], y_point+0.1, 'max', color='red', fontsize=38, ha='center')
    c_plot_image_on_graph(special_x[1],y_point+0.7, torch.tensor(max_latent.reshape(1,4,64,64)).cuda(), ax,'red',3)

    # pca img
    for i, xpos in enumerate(x):
        min_value = file_info_latents[f'cvec_pc{pc_num+1}_dist'][(file_info_latents[f'cvec_pc{pc_num+1}']>xpos-0.015) &  (file_info_latents[f'cvec_pc{pc_num+1}']< xpos+0.015)].min()
        point = file_info_latents['avec'][file_info_latents[f'cvec_pc{pc_num+1}_dist']== min_value].values[0] 
        plt.subplots_adjust(top=0.9)
        c_plot_image_on_graph(xpos,y_point-0.5, torch.Tensor(point.reshape(1,4,64,64)).cuda(), ax,'black',1)

    # 그림 정보 저장
    # min, munch, pearl, max
    # (min) point, title, author, year, (max) point, title, author, year , munch, pearl
    temp_max = file_info_latents[file_info_latents.index == temp_df_max['index'].iloc[-1]]
    temp_min = file_info_latents[file_info_latents.index == temp_df_min['index'].iloc[0]]
    pc_img_info[f'C_PC{pc_num+1}'] = [min(pca_cvec_df[f'cvec_pc{pc_num+1}']), temp_min['painting_name'].values[0], temp_min['author_name'].values[0], temp_min['new_date'].values[0], max(pca_cvec_df[f'cvec_pc{pc_num+1}']), temp_max['painting_name'].values[0], temp_max['author_name'].values[0], temp_max['new_date'].values[0],
                                    pca_cvec_df[f'cvec_pc{pc_num+1}'][munch.index.values[0]], pca_cvec_df[f'cvec_pc{pc_num+1}'][pearl.index.values[0]]]


    plt.title(f'PC{pc_num+1}',fontsize=45, pad=10)
    plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/figure02_D_pc{pc_num+1}.svg',bbox_inches='tight',transparent = True,dpi=600)

    plt.show()


In [ ]:
C_vector_PC_image_info = pd.DataFrame(pc_img_info).T
C_vector_PC_image_info.columns = ['min_point', 'min_title', 'min_author', 'min_date', 'max_point', 'max_title', 'max_author', 'max_date', 'munch', 'pearl']

In [ ]:
C_vector_PC_image_info

In [ ]:
pd.concat([A_vector_PC_image_info,C_vector_PC_image_info]).to_csv('/home/jinny/projects/Art-history/Art-history/graph/datas/pc_point_info.csv')